In [228]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

In [230]:
db = pd.read_excel("Retail-Supply-Chain-Sales-Dataset.xlsx")

In [231]:
db.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Retail Sales People,Product ID,Category,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Cassandra Brandow,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,Not,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-08-11,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,Cassandra Brandow,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Not,731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-12-06,2016-12-06,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,Anna Andreadi,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,Not,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-11-10,2015-11-10,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Cassandra Brandow,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,Not,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-11-10,2015-11-10,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,Cassandra Brandow,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,Not,22.3680,2,0.20,2.5164


In [232]:
db.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 23 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Row ID               9994 non-null   int64         
 1   Order ID             9994 non-null   object        
 2   Order Date           9994 non-null   datetime64[ns]
 3   Ship Date            9994 non-null   datetime64[ns]
 4   Ship Mode            9994 non-null   object        
 5   Customer ID          9994 non-null   object        
 6   Customer Name        9994 non-null   object        
 7   Segment              9994 non-null   object        
 8   Country              9994 non-null   object        
 9   City                 9994 non-null   object        
 10  State                9994 non-null   object        
 11  Postal Code          9994 non-null   int64         
 12  Region               9994 non-null   object        
 13  Retail Sales People  9994 non-nul

In [233]:
target = db.iloc[:,[4,6,7,9] + list(range(16,22))].copy()

In [234]:
target.sample(200)

,Ship Mode,Customer Name,Segment,City,Sub-Category,Product Name,Returned,Sales,Quantity,Discount
5574,Standard Class,Erin Ashbrook,Corporate,Columbus,Phones,Lunatik TT5L-002 Taktik Strike Impact Protecti...,Yes,36.738,1,0.4
6382,Standard Class,Troy Blackwell,Consumer,Springfield,Phones,Clearsounds A400,Not,158.376,4,0.4
6839,Standard Class,Fred Chung,Corporate,Torrance,Phones,Geemarc AmpliPOWER60,Not,371.200,5,0.2
2020,First Class,Rick Huthwaite,Home Office,Providence,Storage,"Hot File 7-Pocket, Floor Stand",Not,1606.230,9,0.0
9088,Standard Class,Rick Reed,Corporate,Cary,Phones,Cisco Small Business SPA 502G VoIP phone,Not,246.168,3,0.2
...,...,...,...,...,...,...,...,...,...,...
8665,First Class,Heather Jas,Home Office,San Francisco,Supplies,Acme Tagit Stainless Steel Antibacterial Scissors,Yes,49.500,5,0.0
8587,Standard Class,Karen Bern,Corporate,Los Angeles,Appliances,Fellowes 8 Outlet Superior Workstation Surge P...,Yes,168.100,5,0.0
1875,Standard Class,Caroline Jumper,Consumer,Long Beach,Bookcases,O'Sullivan Living Dimensions 5-Shelf Bookcases,Not,176.784,1,0.2
7590,Standard Class,Dean Braden,Consumer,Philadelphia,Furnishings,"6"" Cubicle Wall Clock, Black",Not,58.248,9,0.2


In [235]:
target['Duration'] =db['Ship Date']-db['Order Date']

In [242]:
target.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype          
---  ------         --------------  -----          
 0   Ship Mode      9994 non-null   object         
 1   Customer Name  9994 non-null   object         
 2   Segment        9994 non-null   object         
 3   City           9994 non-null   object         
 4   Sub-Category   9994 non-null   object         
 5   Product Name   9994 non-null   object         
 6   Returned       9994 non-null   object         
 7   Sales          9994 non-null   float64        
 8   Quantity       9994 non-null   int64          
 9   Discount       9994 non-null   float64        
 10  Duration       9994 non-null   timedelta64[ns]
dtypes: float64(2), int64(1), object(7), timedelta64[ns](1)
memory usage: 859.0+ KB


In [244]:
target.sample(20)

,Ship Mode,Customer Name,Segment,City,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Duration
2925,First Class,Justin MacKendrick,Consumer,Watertown,Furnishings,"Tenex Carpeted, Granite-Look or Clear Contempo...",Not,70.710,1,0.0,89 days
3905,Standard Class,Valerie Takahito,Home Office,Philadelphia,Binders,Cardinal Slant-D Ring Binders,Not,10.428,4,0.7,153 days
9146,Standard Class,Claudia Bergmann,Corporate,Burlington,Accessories,Logitech Wireless Marathon Mouse M705,Not,99.980,2,0.0,4 days
7281,Standard Class,Gary Hansen,Home Office,Yuma,Machines,Swingline SM12-08 MicroCut Jam Free Shredder,Not,599.985,5,0.7,5 days
9872,Same Day,Mark Hamilton,Consumer,Chicago,Art,Staples in misc. colors,Not,19.152,2,0.2,0 days
7419,Second Class,Stewart Carmichael,Corporate,Philadelphia,Storage,"SAFCO Commercial Wire Shelving, Black",Not,552.560,5,0.2,2 days
2465,Standard Class,Chuck Sachs,Consumer,New York City,Labels,Avery 520,Not,3.150,1,0.0,184 days
3803,Standard Class,Nathan Cano,Consumer,Jacksonville,Supplies,Staple remover,Not,12.768,2,0.2,6 days
732,Standard Class,Darrin Sayre,Home Office,Seattle,Storage,Carina Double Wide Media Storage Towers in Nat...,Not,242.940,3,0.0,4 days
7135,Standard Class,Tonja Turnell,Home Office,Richmond,Chairs,SAFCO Arco Folding Chair,Not,828.600,3,0.0,0 days


In [246]:
target['Returned']=db['Returned'].map({'Yes':1,'Not':0})
def extract_brand(product_name):
    if isinstance(product_name,str):
        return product_name.split()[0].strip(",").title()
    return "Unknown"
target['Brand']=target['Product Name'].apply(extract_brand)
target['Duration']=target['Duration'].dt.days

In [248]:
target.sample(10)

,Ship Mode,Customer Name,Segment,City,Sub-Category,Product Name,Returned,Sales,Quantity,Discount,Duration,Brand
9972,Standard Class,Ryan Crowe,Consumer,Houston,Envelopes,Cameo Buff Policy Envelopes,0,99.568,2,0.2,0,Cameo
7127,Standard Class,Justin MacKendrick,Consumer,Newark,Paper,Tops Wirebound Message Log Books,0,16.450,5,0.0,6,Tops
751,Standard Class,Roland Fjeld,Consumer,Trenton,Art,Avery Hi-Liter EverBold Pen Style Fluorescent ...,0,56.980,7,0.0,120,Avery
1718,Standard Class,Bradley Nguyen,Consumer,New York City,Furnishings,Nu-Dell Float Frame 11 x 14 1/2,0,35.920,4,0.0,5,Nu-Dell
1040,Standard Class,Robert Dilbeck,Home Office,Saint Peters,Tables,Chromcraft Round Conference Tables,0,697.160,4,0.0,4,Chromcraft
484,First Class,Michelle Tran,Home Office,Los Angeles,Labels,Alphabetical Labels for Top Tab Filing,1,29.600,2,0.0,0,Alphabetical
5735,Standard Class,Jim Karlsson,Consumer,New York City,Binders,Wilson Jones Clip & Carry Folder Binder Tool f...,0,13.920,3,0.2,0,Wilson
908,Standard Class,Patrick O'Donnell,Consumer,New York City,Binders,Wilson Jones Legal Size Ring Binders,0,52.776,3,0.2,61,Wilson
477,Standard Class,Kristen Hastings,Corporate,Los Angeles,Phones,Cisco SPA525G2 IP Phone - Wireless,0,95.760,6,0.2,0,Cisco
2811,Second Class,Mike Pelletier,Home Office,Milwaukee,Furnishings,Advantus Employee of the Month Certificate Fra...,0,185.580,6,0.0,2,Advantus


In [250]:
encoding_cols = ['Ship Mode','Segment','City','Sub-Category','Brand']
for cols in encoding_cols:
    le=LabelEncoder()
    le.fit_transform(target[cols])
    target[cols]=le.transform(target[cols])
target.drop(columns=['Product Name','Customer Name'],inplace= True)


In [252]:
target.sample(10)

,Ship Mode,Segment,City,Sub-Category,Returned,Sales,Quantity,Discount,Duration,Brand
7323,0,0,329,9,0,41.960,2,0.0,92,263
4054,3,1,374,12,0,20.928,4,0.2,0,390
3593,3,2,485,3,0,22.428,3,0.7,0,170
8598,3,1,159,3,0,4.320,3,0.2,0,455
6836,3,1,482,9,0,22.720,1,0.0,5,158
7687,3,1,215,3,0,17.300,1,0.0,4,455
9226,1,2,55,14,0,52.752,3,0.2,0,354
5573,3,1,94,12,1,17.472,3,0.2,123,72
1177,3,0,414,5,0,172.500,2,0.0,121,188
1231,2,0,56,2,0,2.780,2,0.0,0,131
